In [ ]:
using Plots

Many of our numerical differentiation and integration methods take the form
$$
\text{Absolute Error at $h$} = E(h) =  C h^p + \mathrm{O}(h^{p+1}),
$$
so that to leading order $E(h) \approx C h^p$.  $p$ is the **order** of the method.  We can estimate this from data by noting:
$$
\ln E(h) \approx \ln C + p \ln (h),
$$
so that on $\log-\log$ plot, $p$ will be the slope of the line.  This $p$ can then be estimated by linear regression methods.

# Example
Recall that if $f(x) = \arctan(x)$, $f'(x) = 1/(1+x^2)$.  Using the centered midpoint formula, we will estimate $f'(1)=0.5$ at a series of values of $h$ and try to recover the theoretical $p=2$ order of convergence. 

In [ ]:
f = x->atan(x);
x0 = 1;
h_vals = 0.1 * 2. .^(0:-1:-4);
df_vals = [];
err_vals = [];
for h in h_vals
    df = (f(x0+h)-f(x0-h))/(2*h); # 3 point centered difference scheme
    push!(df_vals, df);
    err = abs(df -0.5); # compute absolute error
    push!(err_vals, err);
    println("h = $(round(h, digits=5)), df = $df, Abs. Err. = $err")
end
scatter(h_vals, err_vals, xscale=:log10, yscale=:log10, label="Data", xlabel="h", ylabel="Abs. Error")

Let us compare, empirically with several values of $p$.   We empircally put the values 0.01 and 0.1 to get the curves to be visually helpful. 

In [ ]:
 
scatter(h_vals, err_vals, xscale=:log10, yscale=:log10, label="Data", xlabel="h", ylabel="Abs. Error", legend=:bottomright)
plot!(h_vals, 0.01 * h_vals, lw=2, label="h", line=:dash)
plot!(h_vals, 0.1 * h_vals.^2, lw=2, label="h^2", line=:dash)
plot!(h_vals, h_vals.^3, lw=2, label="h^3", line=:dash)

It lines up well with $\propto h^2$, which is what we know about the method from previous lectures.

We can be a bit more systematic by performing a linear regression.  This requires packaging our data into a `DataFrame` and then using `GLM` to perform the regression:

In [ ]:
using DataFrames, GLM

In [ ]:
data = DataFrame(y = log.(err_vals), x = log.(h_vals))
model = lm(@formula(y ~ x), data) # fits a linear model of y = c0 + c1 x


This finds a slope of $1.9903\approx 2$.

# Estimating the order of convergence without the true value
Now, we will use our highest precision result, stored in `df_vals[end]`, as a surrogate for the true value, and estimate the order with that data.

In [ ]:
est_err = abs.(df_vals[1:end-1] .- df_vals[end]); #estimate the error using highest precision result as a surrogate for "exact"

scatter(h_vals[1:end-1], est_err, xscale=:log10, yscale=:log10, label="Data", xlabel="h", ylabel="Est. Error")
plot!(h_vals, 0.01 * h_vals, lw=2, label="h", line=:dash)
plot!(h_vals, 0.1 * h_vals.^2, lw=2, label="h^2", line=:dash)
plot!(h_vals, h_vals.^3, lw=2, label="h^3", line=:dash)

Again, it looks quadratic.

In [ ]:
data = DataFrame(y = log.(est_err), x = log.(h_vals[1:end-1]))
model = lm(@formula(y ~ x), data)


This (`2.12849`) is not quite as good , but it certainly suggestive of quadratic convergence.